In [60]:
from lmfit import minimize, Parameters, Parameter, report_fit
import math
import numpy as np
import pandas as pd

In [11]:
data = pd.read_csv("./augment/softmax.csv", header=None)

In [46]:
total_vals = len(data.iloc[0, :])
sim_size = total_vals - 4
sim_data = data.iloc[:, 0:sim_size]
true_data = data.loc[:, sim_size:sim_size + 4]

In [47]:
for i in range(0, sim_size,  4):
    if i == 0:
        sim_d = sim_data.iloc[:, i].values
    else:
        sim_d = np.concatenate((sim_d, sim_data.iloc[:, i].values), axis=0)
        
for i in range(1, sim_size,  4):
    if i == 1:
        sim_r = sim_data.iloc[:, i].values
    else:
        sim_r = np.concatenate((sim_r, sim_data.iloc[:, i].values), axis=0)
        
for i in range(2, sim_size,  4):
    if i == 2:
        sim_vel = sim_data.iloc[:, i].values
    else:
        sim_vel = np.concatenate((sim_vel, sim_data.iloc[:, i].values), axis=0)
        
for i in range(3, sim_size,  4):
    if i == 3:
        sim_acc = sim_data.iloc[:, i].values
    else:
        sim_acc = np.concatenate((sim_acc, sim_data.iloc[:, i].values), axis=0)

In [49]:
# Get true values
true_d = true_data.iloc[:, 0].values
true_r = true_data.iloc[:, 1].values
true_vel = true_data.iloc[:, 2].values
true_acc = true_data.iloc[:, 3].values

In [92]:
acc = np.zeros(len(true_acc))
x = np.zeros((len(true_acc), 3))

count = 0
for i in range(1, len(true_acc)):
    if true_acc[i] != 0 and (i - 1) % 100 != 0 and true_d[i-1] != 0:
        acc[count] = true_acc[i]
        x[count, :] = [true_d[i-1], true_r[i-1], true_vel[i-1]]
        count += 1
        
acc = acc[:count]
x = x[:count, :]

In [95]:
# Function to minimize
def IDM(params, i, true_acc):
    dmn = params['dmn'].value
    T = params['T'].value
    bcmf = params['bcmf'].value
    smx = params['smx'].value
    
    amx = 3
    d = x[i, 0]
    r = x[i, 1]
    s = x[i, 2]
    
    d_des = dmn + T*s - s*r/2/math.sqrt(amx*bcmf)
    pred = amx*(1 - (s/smx)**4 - (d_des/d)**2)
    
    return abs(pred - true_acc)
    

In [143]:
# Define parameters
params = Parameters()
params.add('dmn', value = 15.0, min = 0.1, max = 20)
params.add('T', value = 3., min = 1.0, max = 10)
params.add('bcmf', value = 5.0, min = 0.1, max = 5)
params.add('smx', value = 17.0, min = 0.1, max = 40)

In [144]:
result = minimize(IDM, params, args=(range(len(acc)), acc))

In [145]:
result.params

Parameters([('dmn', <Parameter 'dmn', value=5.2723398878751597 +/- 0.0534, bounds=[0.1:20]>), ('T', <Parameter 'T', value=1.0001891519558783 +/- 0.00791, bounds=[1.0:10]>), ('bcmf', <Parameter 'bcmf', value=4.8734758643340692 +/- 0.058, bounds=[0.1:5]>), ('smx', <Parameter 'smx', value=17.169163489759775 +/- 0.066, bounds=[0.1:40]>)])

In [146]:
sum(result.residual)/count

1.5426901532820234